In [2]:
from google.colab import drive
drive.mount('/content/drive')

!pip install folium==0.2.1
!pip install tensorflow==2.8
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2
!pip install -q tflite-model-maker
!pip install -q tflite-support

import numpy as np
import os

from tflite_model_maker.config import ExportFormat, QuantizationConfig
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

from tflite_support import metadata

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

In [3]:
!unzip -q "$PROJECT_DIR/training_set.zip"

In [7]:
import json
with open(os.path.join(PROJECT_DIR, f"{PROJECT_NAME}.json")) as file:
    info = json.load(file)

train_data = object_detector.DataLoader.from_pascal_voc(
    '/content/train',
    '/content/train',
    info['classes']
)

val_data = object_detector.DataLoader.from_pascal_voc(
    '/content/validate',
    '/content/validate',
    info['classes']
)

In [6]:
spec = model_spec.get('efficientdet_lite0')
model = object_detector.create(train_data, model_spec=spec, batch_size=4, train_whole_model=True, epochs=20, validation_data=val_data)

In [ ]:
model.evaluate(val_data)

In [ ]:
model.export(export_dir='.', tflite_filename=f'{PROJECT_NAME}.tflite')

In [ ]:
model.evaluate_tflite(f'{PROJECT_NAME}.tflite', val_data)

In [ ]:
!cp "$PROJECT_NAME"'.tflite' "$PROJECT_DIR"